In [1]:
!pip install opencv-python-headless
!pip install mediapipe



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_detection



# Initialize Face Detection and Pose Estimation models
face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)
pose_model = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)


I0000 00:00:1701260999.970424       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1701260999.981595       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2 Pro


In [4]:

cap = cv2.VideoCapture(0)


In [ ]:
# https://www.youtube.com/watch?v=FsVAvgR9ifY
while True:

    ret, frame = cap.read()

    if not ret:
        continue

    # Convert to rgb because if not movenet does not capture it. ask lorcan about this bad boy
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_results = face_detection.process(frame_rgb)
    results = pose_model.process(frame_rgb)

# results.(pose_landmarks, pose_world_landmarks, segmentation_mask)
    if results.pose_landmarks:   # could be multi_hand_landmarks
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=10, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=10, circle_radius=2),
)

# mp_pose is mp.solutions.pose. POSE_CONNECTIONS is part of doc only option

    if face_results.detections:
        for detection in face_results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # mp_drawing.draw_landmarks(image: numpy.ndarray,
#     landmark_list: mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList,
#     connections: Optional[List[Tuple[int, int]]] = None,
#     landmark_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[int, mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
#     connection_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2),
# )


    cv2.imshow('Body Detector', frame)


    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
#kill cam from cv2 when break


In [ ]:
## this code has both but bad pose estimation. therefore implement this code from stack onto mine above
##this code also does the body based on face refcognition which is something we dont want, as the face might not be visible from side perspective
## https://stackoverflow.com/questions/77426154/how-to-find-the-face-orientation-using-mediapipe

# while True:
#     success, frame = cap.read()
#     if not success:
#         continue

#     # Convert the frame to RGB format for MediaPipe
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Perform face detection
#     face_results = face_detection.process(frame_rgb)

#     if face_results.detections:
#         for detection in face_results.detections:
#             bboxC = detection.location_data.relative_bounding_box
#             ih, iw, _ = frame.shape
#             x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

#             # Perform pose estimation using the face region
#             face_center = (x + w // 2, y + h // 2)
#             frame_pose = frame[y:y + h, x:x + w]
#             frame_pose_rgb = cv2.cvtColor(frame_pose, cv2.COLOR_BGR2RGB)

#             # Perform pose estimation on the face region
#             pose_results = pose_estimation.process(frame_pose_rgb)

#             if pose_results.pose_landmarks:
#                 # You can access pose landmarks and orientation information here
#                 landmarks = pose_results.pose_landmarks
#                 # Extract relevant pose landmarks and calculate face orientation

#                 # Draw landmarks and orientation lines
#                 mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#     cv2.imshow("Face Orientation Detection", frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
